In [ ]:
!pip install datasets==2.16.1

In [ ]:
import random
import re
from typing import List

from datasets import load_dataset, list_datasets, Dataset
from IPython.display import Image
from IPython.core.display import HTML
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Model
from tqdm import tqdm

https://huggingface.co/docs/transformers/model_doc/gpt2

https://huggingface.co/docs/transformers/main_classes/tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")
print(f"Our device is {DEVICE}")

# Как работает генерация с т.з. кода под капотом:

In [ ]:
TEXT_INPUT = "Парламент- это не место для"

In [ ]:
inputs = tokenizer(TEXT_INPUT, return_tensors="pt")

In [ ]:
for k, v in inputs.items():
  inputs[k] = v.to(DEVICE)

In [ ]:
bare_model = GPT2Model.from_pretrained("gpt2")
bare_model.eval()
bare_model.to(DEVICE)
bare_outputs = bare_model(**inputs, output_hidden_states=True)

In [ ]:
last_hidden_states = bare_outputs.last_hidden_state

Weight tying

In [ ]:
Image(url= "https://lena-voita.github.io/resources/lectures/lang_models/practical/weight_tying_idea-min.png", width=1900, height=900)

In [ ]:
logits = torch.matmul(
    last_hidden_states[-1][-1],
    bare_model.wte.weight.T
)

In [ ]:
bare_probas = F.softmax(logits, dim=0)

In [ ]:
torch.argmax(bare_probas)

# Упрощённая генерация:

In [ ]:
llm_model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
llm_model.eval()
llm_model.to(DEVICE)
llm_outputs = llm_model(**inputs)

In [ ]:
torch.argmax(llm_outputs.logits[-1][-1])

In [ ]:
llm_probas = F.softmax(llm_outputs.logits[-1][-1], dim=0)

In [ ]:
torch.allclose(bare_probas, llm_probas, rtol=1e-4)

# Задание 1: написать свою имплементацию BPE

Как работает алгоритм: \
- У алгоритма один гиперпараметр- число итераций M \
- На  i-ой итерации(1 <= i <= M) мы находим самую популярную пару токенов, идущих подряд \(для примера наховём их a, b) \
- Мы создаём новый токен, соответствующий конкатенции пары из предыдущего пункта \(a,b -> ab), все вхождения пары в тренировочных данных заменяем на новый токен
- Возвращаемся к шагу 2

In [ ]:
training_data = [
    "a", "b", "c", "d",
    "e",
    "a",
    "a", "b", "c", "d",
    "b", "c", "d"
]

In [ ]:
ALPHABET = set(training_data)

In [ ]:
print(f"This is my alphabeth: {ALPHABET}")
print()
print(f"Its length is {len(ALPHABET)}")

In [ ]:
NUM_MERGES = 3

In [ ]:
"""
Your implementation here
"""

In [ ]:
print(f"This is my alphabeth: {ALPHABET}")
print()
print(f"Its length is {len(ALPHABET)}")

# Задание 2: написать fine-tuning для языковой модели под набор данных:

Описание датасета можно найти тут: \
https://paperswithcode.com/dataset/rucos \
https://huggingface.co/datasets/RussianNLP/russian_super_glue

In [ ]:
dataset = load_dataset("RussianNLP/russian_super_glue", name='rucos')

In [ ]:
RE_BAD_PATTERNS = re.compile("(@[a-z]+|\n)")

In [ ]:
random_idx_from_train = random.randint(0, len(dataset['train']) - 1)

random_object = dataset['train'][random_idx_from_train]['passage']

filtered_random_object = RE_BAD_PATTERNS.sub(" ", random_object)
print(random_object)
print("*" * 20)
print(filtered_random_object)

In [ ]:
tokenizer.add_special_tokens({'pad_token': "<|endoftext|>"})

def texts_to_batch(texts: List[str]) -> torch.Tensor:
    clean_texts = [
        RE_BAD_PATTERNS.sub(" ", _["passage"]) for _ in texts
    ]
    batch = tokenizer(
        text=clean_texts,
        return_tensors="pt",
        add_special_tokens=True,
        padding="max_length",
        truncation=True
      )
    return batch

In [ ]:
BATCH_SIZE = 2

train_dl = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=texts_to_batch
)

In [ ]:
N_ITERATIONS = 1000

inputs = tokenizer("В прошлый четверг президенты Казахстана и России", return_tensors="pt")
for k, v in inputs.items():
  inputs[k] = v.to(DEVICE)

In [ ]:
OUTPUT_SIZE = 40

In [ ]:
llm_model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
llm_model.eval()
llm_model.to(DEVICE)
llm_outputs = llm_model(**inputs)

Подумайте о том:
- как формировать входной батч токенов и attention_masks для модели
- как формировать батч лейблов
- как будете считать лосс

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=llm_model.parameters(), lr=1e-6
)

cur_iteration = 0
for batch in train_dl:
    if cur_iteration == N_ITERATIONS:
      break

    llm_model.train()
    """
    Your implementation here
    """
    loss_value = None  # Write your loss computation here and after each batch print it to ensure that the loss decreases
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()


    """
    After the batch consumption, see how your model performs to ensure that it's getting better at generation
    """
    llm_model.eval()
    for n_beams in range(2, 5):
      beam_output = llm_model.generate(**inputs, max_new_tokens=OUTPUT_SIZE, num_beams=n_beams)
      print(f"Beam size={n_beams}")
      print(tokenizer.decode(beam_output[0], skip_special_tokens=True))
      print()

    print("*" * 20)
    cur_iteration += 1

# Задание 3: написать greedy search, сравнить результаты с имплементацией от transformers

In [ ]:
OUTPUT_SIZE = 40

In [ ]:
llm_model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
llm_model.eval()
llm_model.to(DEVICE)
llm_outputs = llm_model(**inputs)

In [ ]:
def convert_to_expected_input(input_ids, attention_mask):
    input_ids = torch.tensor(input_ids, device=DEVICE)
    attention_mask = torch.tensor(attention_mask, device=DEVICE)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }

Напомним, как работает greeedy search:
- на каждой итерации i по последовательности токенов s мы делаем предсказание и берём максимальный по вероятности токен
- он конкатенируется к s
- возвращаемся в п.1

In [ ]:
"""
Note, that input_ids and attention_mask are LISTS!
input_ids should contain the tokens you predict after each iteration
"""
input_ids, attention_mask = inputs["input_ids"].tolist(), inputs["attention_mask"].tolist()

for _ in range(OUTPUT_SIZE):
  """
  Your implementation here
  """

In [ ]:
llm_predictions = llm_model.generate(**inputs, max_new_tokens=OUTPUT_SIZE)

Ниже проверяем, что наивная имплементация совпадает с ожидаемой:

In [ ]:
assert input_ids[-1][-OUTPUT_SIZE:] == llm_predictions[-1][-OUTPUT_SIZE:].tolist()

In [ ]:
tokenizer.decode(llm_predictions[0], skip_special_tokens=True)